In [ ]:
#|default_exp export

In [ ]:
#|export
from __future__ import annotations

# PyTest Capabilities with nbprocess

> A notebook allowing for the export of tests in an nbprocess project and thrown into a pytest-compatible file 

In [ ]:
#|export
from nbprocess.process import *
from nbprocess.read import *
from nbprocess.imports import *
from nbprocess.maker import *

from nbprocess.processors import _default_exp

In [ ]:
_test_file = "00_core.ipynb"

The goal of this notebook is to demonstrate how easy it can be to add automatic export of tests written inside of a Jupyter Notebook through `nbprocess`.

What does this look like? I'd recommend reading through [00_core](https://github.com/muellerzr/nbprocess-sandbox/blob/main/00_core.ipynb) first to get an idea, but essentially we write one big `unittest.TestCase` for each file (which is commonly done), and then we declare each of our tests through a `#|test {testname}` tag. There is also a special `#|test imports` tag for imports that should be present when performing the testing. 

Importing the exported module via `from library.{module} import *` is done automatically.

In [ ]:
#|export
from collections import defaultdict
from fastcore.foundation import L, ifnone
from execnb.nbio import *

class ExportTestProc:
    "A test proc that watches for `#|default_exp` and `#|test`"
    def __init__(self): self.tests = defaultdict(L)
    def _default_exp_(self, nbp, cell, exp_to): self.default_exp = f'test_{exp_to}'
    def _test_(self, nbp, cell, exp_to=None, nm=None, tst_cls=None): self.tests[self.default_exp].append(nbp.cell)

In [ ]:
#|export
_re_test = re.compile(r'#\|\s*test\s*$', re.MULTILINE)
_re_import = re.compile(r'#\|\s*test\s*import\s*$', re.MULTILINE)
_tab = "    "

In [ ]:
#|export
def get_directive(cell, key, default=None): 
    "Extract a top level directive from `cell`"
    return cell.directives_.get(key, default)

def _is_test_cell(cell): return cell.cell_type == "code" and get_directive(cell, "test")

## Using PyTest

In [ ]:
#|export
def _mark_test(s):
    ft = exec_new("import fastcore.test as ft")["ft"].__all__
    kinds = [o for o in ft if o.startswith("test")]
    for k in kinds:
        if f"{k}(" in s: 
            s = s.replace(f"{k}(", f"ft.{k}(")
    return s

In [ ]:
#|export
def convert_pytest(cell):
    "Wraps cell contents into a pytest function"
    directive = get_directive(cell, "test")
    if _is_test_cell(cell):
        if "import" not in directive:
            content = '\n'.join([f"{_tab}{c}" for c in cell.source.split("\n")])
            content = _mark_test(content)
            cell.source = f'def test_{directive[0]}():\n{content}'
        else:
            cell.source = cell.source.replace("from fastcore.test import *", "import fastcore.test as ft")

To use this processor, make sure your cell has the following directive format:
```python
#| test {test_name}
```

Below we can test it out:

In [ ]:
#|export
def construct_imports(nb, use_unittest=False):
    "Generates the test imports for the notebook"
    libname = get_config().lib_name
    exp = _default_exp(nb)
    imports = ['#| test import\n', f'from {libname}.{exp} import *\n']
    if use_unittest: imports += ['import unittest']
    nb.cells.insert(1, mk_cell(imports))

This function will make sure to import the module being exported into your notebook, and potentially include a `unittest` import.

In [ ]:
#|export
def create_test_modules(path,dest,debug=False,mod_maker=ModuleMaker, unittest=False):
    "Creates test files from `path`, optionally with unittest support"
    exp = ExportTestProc()
    procs = [exp, convert_pytest]
    if unittest: procs.append(convert_unittest)
    nb = NBProcessor(path, procs, preprocs=partial(construct_imports, use_unittest=unittest))
    nb.process()
    is_new = True
    for mod,cells in exp.tests.items():
        mm = mod_maker(dest=dest, name=exp.default_exp, nb_path=path, is_new=is_new)
        mm.make(cells)
        is_new = False

In [ ]:
#|eval: false
create_test_modules(_test_file, "tmp")

============================= test session starts ==============================
platform linux -- Python 3.9.7, pytest-7.1.2, pluggy-1.0.0
rootdir: /home/jovyan/local/zach/nbprocess-sandbox
plugins: anyio-3.5.0
collected 1 item                                                               

tmp/test_core.py .                                                       [100%]

============================== 1 passed in 0.01s ===============================


## Using unittest

In [ ]:
#|export
def convert_unittest(cell):
    "Wraps cell contents into a unittest test suite."
    if _is_test_cell(cell):
        directive = get_directive(cell, "test")
        if "case" in directive:
            tstcls, = directive[2:3] or "unittest.TestCase"
            cell.source = f'class {directive[1]}({tstcls}):'
        elif "import" not in directive:
            cell.source = '\n'.join([f'{_tab}{c}' for c in cell.source.split("\n")])

To use this processor, make sure a cell earlier in the notebook has the following directive format:
```python
#|test case {CaseName} {TestCaseClassConstructor}
```

For example:
```python
#|test case SomeTest unittest.TestCase
```
or:
```python
#| test case MyTest SomeCustomTestCaseClass
```

In [ ]:
#|eval: false
create_test_modules(_test_file, "unittest",unittest=True)

g = exec_new("from tmp.test_core import CoreTester")
assert hasattr(g["CoreTester"], "test_addition")

Finally we can run the test case and ensure it passes:

In [ ]:
#|eval: false
import unittest
def run_case(testcase:unittest.TestCase):
    "Runs a unittest.TestCase"
    suite = unittest.defaultTestLoader.loadTestsFromTestCase(testcase)
    unittest.TextTestRunner().run(suite)

In [ ]:
#|eval: false
run_case(g["CoreTester"])

.
----------------------------------------------------------------------
Ran 1 test in 0.002s

OK
